In [15]:
import plotly.graph_objects as go
import numpy as np
from numpy import cos, sin
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display, clear_output
from dataclasses import dataclass

## Calculations 

In [16]:
TABLE_RADIUS = 3
TABLE_ANGLE = np.deg2rad(80)
BASE_RADIUS = 7
BASE_ANGLE = np.deg2rad(15)
MAX_POSSIBLE_HEIGHT = 15
MIN_POSSIBLE_HEIGHT = 0
ARM_LENGTH = 8
### CONSTANTS THAT SHOULD NOT BE CHANGED
NUM_OF_HEXAGON_VERTICES = 6
NUM_OF_TRANSFORMED_VECTORS = NUM_OF_HEXAGON_VERTICES + 2
NUM_OF_SLIDERS = 6
###

### CONSTANTS THAT CAN BE CHANGED
NUM_OF_HOMOTOPY_FRAMES = 3;

NEEDLE_LENGTH = 3
TABLE_RADIUS = 3
TABLE_ANGLE = np.deg2rad(80)
BASE_RADIUS = 7
BASE_ANGLE = np.deg2rad(15)
MAX_POSSIBLE_HEIGHT = 15
MIN_POSSIBLE_HEIGHT = 0
ARM_LENGTH = 8
###


@dataclass
class NeedleCoordinates:
    x: float
    y: float
    z: float
    phi: float
    theta: float
    psi: float
    
def populate_transformation_matrix( c :NeedleCoordinates, homotopy_coefficient: float):
    assert homotopy_coefficient <= 1.0 and homotopy_coefficient >= 0.0
    x = c.x
    y = c.y
    z = c.z
    phi = c.phi
    theta = c.theta
    psi = c.psi
    transformation_matrix = np.array([
        [cos(psi)*cos(theta), cos(psi)*sin(theta)*sin(phi)-sin(psi)*cos(phi), cos(psi)*sin(theta)*cos(phi)+sin(psi)*sin(phi), x],
        [sin(psi)*cos(theta), sin(psi)*sin(theta)*sin(phi)+cos(psi)*cos(phi), sin(psi)*sin(theta)*cos(phi)-cos(psi)*sin(phi), y],
        [-sin(theta), cos(theta)*sin(phi), cos(theta)*cos(phi), z],
        [0, 0, 0, 1]
    ])
    identity_matrix = np.array( [
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1] 
    ] )
    homotopy_frame = (1.0 - homotopy_coefficient)*identity_matrix + homotopy_coefficient * transformation_matrix
    return homotopy_frame
         


def populate_table_3dmodel():
    radius = TABLE_RADIUS
    angle_in_radians = TABLE_ANGLE
    distance_from_needle_end = NEEDLE_LENGTH
    table_3dmodel = np.array([
        [radius * np.sin(np.deg2rad(60)*0+angle_in_radians/2), radius * np.cos(np.deg2rad(60)*0+angle_in_radians/2), distance_from_needle_end],
        [radius * np.sin(np.deg2rad(60)*0-angle_in_radians/2), radius * np.cos(np.deg2rad(60)*0-angle_in_radians/2), distance_from_needle_end],
        [radius * np.sin(np.deg2rad(60)*2+angle_in_radians/2), radius * np.cos(np.deg2rad(60)*2+angle_in_radians/2), distance_from_needle_end],
        [radius * np.sin(np.deg2rad(60)*2-angle_in_radians/2), radius * np.cos(np.deg2rad(60)*2-angle_in_radians/2), distance_from_needle_end],
        [radius * np.sin(np.deg2rad(60)*4+angle_in_radians/2), radius * np.cos(np.deg2rad(60)*4+angle_in_radians/2), distance_from_needle_end],
        [radius * np.sin(np.deg2rad(60)*4-angle_in_radians/2), radius * np.cos(np.deg2rad(60)*4-angle_in_radians/2), distance_from_needle_end],
        [0.0, 0.0, distance_from_needle_end],
        [0.0, 0.0, 0.0]
    ])
    return table_3dmodel
def populate_hexagon(radius, angle_in_radians):
    points = np.array([
        [radius * np.sin(np.deg2rad(60)*0+angle_in_radians/2), radius * np.cos(np.deg2rad(60)*0+angle_in_radians/2), 0],
        [radius * np.sin(np.deg2rad(60)*0-angle_in_radians/2), radius * np.cos(np.deg2rad(60)*0-angle_in_radians/2), 0],
        
        [radius * np.sin(np.deg2rad(60)*2+angle_in_radians/2), radius * np.cos(np.deg2rad(60)*2+angle_in_radians/2), 0],
        [radius * np.sin(np.deg2rad(60)*2-angle_in_radians/2), radius * np.cos(np.deg2rad(60)*2-angle_in_radians/2), 0],
        
        [radius * np.sin(np.deg2rad(60)*4+angle_in_radians/2), radius * np.cos(np.deg2rad(60)*4+angle_in_radians/2), 0],
        [radius * np.sin(np.deg2rad(60)*4-angle_in_radians/2), radius * np.cos(np.deg2rad(60)*4-angle_in_radians/2), 0],
    ])
    
    return points

def transform_vector(vec, transformation_matrix):
    vec_copy = np.array(vec) 
    return np.append(vec_copy, 1).dot(transformation_matrix.T)[:3]

def find_intersection(table_transformed_point, base_point):
    # We calculate intersection points of a vertical line with a sphere. 
    # Sphere equation: (x-table_transformed_point_x)**2 + (y-table_transformed_point_y)**2 + (z-table_transformed_point_x)**2 = ARM_LENGTH**2
    # Vertical line equation: x = base_point_x; y = base_point_y; z is changing.
    table_transformed_point_x, table_transformed_point_y, table_transformed_point_z = table_transformed_point
    base_point_x, base_point_y, _ = base_point
    
    z_part = ARM_LENGTH**2 - (base_point_x - table_transformed_point_x)**2 - (base_point_y - table_transformed_point_y)**2
    if z_part > 0:
        z_coordinate_of_slider = table_transformed_point_z + np.sqrt(z_part) # One of two solutions
        if MAX_POSSIBLE_HEIGHT > z_coordinate_of_slider > MIN_POSSIBLE_HEIGHT:
            return (base_point_x, base_point_y, z_coordinate_of_slider)
    return None

In [17]:
initial_table_points = populate_table_3dmodel()
base_points = populate_hexagon(BASE_RADIUS, BASE_ANGLE)

## Plots and visualisation

In [18]:
AXIS_RANGE = [-15, 15] 
MESH_INDICES = [(0, 1, 6), (2, 3, 6), (4, 5, 6), (1, 4, 6), (3, 0, 6), (5, 2, 6)]
STEP_SIZE = 0.0000001

x_widget = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='X Position')
y_widget = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='Y Position')
z_widget = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='Z Position')
phi_widget = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='Phi Angle')
theta_widget = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='Theta Angle')
psi_widget = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='Psi Angle')
homotopy_widget = widgets.FloatSlider(value=0, min=0, max=1, step=STEP_SIZE, description='Homotopy coefficient')
text_widget = widgets.Textarea(
    value='',
    placeholder='Texbox for intersections',
    disabled=False,
    layout={'width': '320px', 'height': '200px'}
)

WIDGETS = [x_widget, y_widget, z_widget, phi_widget, theta_widget, psi_widget, homotopy_widget]

def find_element_by_tag(fig, tag):
    for i, plot_element in enumerate(fig.data):
        if plot_element.customdata and plot_element.customdata[0] == tag:
            return i
    return None

def format_intersection_points(intersection_points):
    all_positions_found = all(point is not None for point in intersection_points)

    if all_positions_found:
        formatted_intersections = ["Targeted table coordinates can be realized using these slider positions:\n\n"]
        for i, point in enumerate(intersection_points):
            formatted_intersections.append(f"{i}: ({point[0]:.6f}, {point[1]:.6f}, {point[2]:.6f})")
    else:
        formatted_intersections = ["Failed to find slider positions that can realize targeted table coordinates:\n"]
        for i, point in enumerate(intersection_points):
            status = "ok" if point is not None else "None"
            formatted_intersections.append(f"{i}: {status}")

    return '\n'.join(formatted_intersections)


def initialize_plot():
    fig = go.FigureWidget()
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=AXIS_RANGE),
            yaxis=dict(range=AXIS_RANGE),
            zaxis=dict(range=AXIS_RANGE)
        ),
    title='6-DOF robot with vertical parallel rails')
    fig.add_scatter3d(x=initial_table_points[:, 0], y=initial_table_points[:, 1], z=initial_table_points[:, 2], mode='markers', marker=dict(size=5), line=dict(color='blue'), name='Initial Table', customdata=["original_table"])
    fig.add_scatter3d(x=initial_table_points[:, 0], y=initial_table_points[:, 1], z=initial_table_points[:, 2], mode='markers', marker=dict(size=5), line=dict(color='red'), name='Transformed Table', customdata=["transformed_table"])

    for point in base_points:
        fig.add_scatter3d(x=[point[0], point[0]], y=[point[1], point[1]], z=[MIN_POSSIBLE_HEIGHT, MAX_POSSIBLE_HEIGHT], mode='lines', line=dict(color='green', width=2), showlegend=False)

    needle_x = x_widget.value
    needle_y = y_widget.value
    needle_z = z_widget.value
    centroid_x = initial_table_points[6, 0] 
    centroid_y = initial_table_points[6, 1] 
    centroid_z = initial_table_points[6, 2] 
    fig.add_scatter3d(x=[needle_x, centroid_x], y=[needle_y, centroid_y], z=[needle_z, centroid_z], mode='lines', line=dict(color='red', width=2), showlegend=False, customdata=["initial_needle"])

    fig.add_mesh3d(
        x=initial_table_points[:, 0], 
        y=initial_table_points[:, 1], 
        z=initial_table_points[:, 2], 
        i=[i[0] for i in MESH_INDICES], 
        j=[i[1] for i in MESH_INDICES], 
        k=[i[2] for i in MESH_INDICES], 
        color='red', 
        opacity=0.5,
        customdata=["table_mesh"]
    )
    
    for i, (t, b) in enumerate(zip(initial_table_points, base_points)):
        intersection = find_intersection(t, b)
        if intersection:
            _, _, intersection_z = intersection
            fig.add_scatter3d(x=[t[0], b[0]], y=[t[1], b[1]], z=[t[2], intersection_z], mode='lines', line=dict(color='black', width=2), showlegend=False, customdata=[f"arm_{i}"])

    fig.update_layout(scene=dict(xaxis=dict(range= AXIS_RANGE, title='X Axis'), yaxis=dict(range= AXIS_RANGE, title='Y Axis'), zaxis=dict(range= AXIS_RANGE, title='Z Axis'), aspectmode='cube'), margin=dict(l=0, r=0, b=0, t=0))
    return fig

def update_plot(change):
    with plot.batch_update():
        c = NeedleCoordinates(x = x_widget.value, y = y_widget.value, z = z_widget.value, phi = phi_widget.value, theta =theta_widget.value, psi = psi_widget.value)
        homotopy_coefficient = homotopy_widget.value
        transformation_matrix = populate_transformation_matrix(c, homotopy_coefficient)
        transformed_table_points = np.array([transform_vector(point, transformation_matrix) for point in initial_table_points])

        transformed_table = find_element_by_tag(plot, "transformed_table")
        mesh = find_element_by_tag(plot, "table_mesh")
        plot.data[transformed_table].x, plot.data[transformed_table].y, plot.data[transformed_table].z = transformed_table_points.T
        plot.data[mesh].x, plot.data[mesh].y, plot.data[mesh].z = transformed_table_points.T
        plot.data[mesh].i, plot.data[mesh].j, plot.data[mesh].k = zip(*MESH_INDICES)


        needle = find_element_by_tag(plot, "initial_needle")
        plot.data[needle].x = [transformed_table_points[6][0], transformed_table_points[7][0]]
        plot.data[needle].y = [transformed_table_points[6][1], transformed_table_points[7][1]]
        plot.data[needle].z = [transformed_table_points[6][2], transformed_table_points[7][2]]

        intersection_points = []
        for i, (t, b) in enumerate(zip(transformed_table_points, base_points)):
            intersection = find_intersection(t, b)
            arm = find_element_by_tag(plot, f"arm_{i}")

            if intersection:
                intersection_points.append(intersection)
                _, _, intersection_z = intersection
                plot.data[arm].x = [t[0], b[0]]
                plot.data[arm].y = [t[1], b[1]]
                plot.data[arm].z = [t[2], intersection_z]
            else:
                intersection_points.append(None)
                plot.data[arm].x = [None, None]
                plot.data[arm].y = [None, None]
                plot.data[arm].z = [None, None]

        intersection_text = format_intersection_points(intersection_points)
        text_widget.value = intersection_text

In [19]:
plot = initialize_plot()
for widget in WIDGETS:
    widget.observe(update_plot, names='value')

In [20]:
x_widget.value = 0.5
y_widget.value = 0.5
z_widget.value = 3.0
phi_widget.value = -np.pi/10
theta_widget.value = np.pi/10
psi_widget.value = np.pi/10
homotopy_widget.value = 1.0

In [21]:
display(plot)

FigureWidget({
    'data': [{'customdata': [original_table],
              'line': {'color': 'blue'},
              'marker': {'size': 5},
              'mode': 'markers',
              'name': 'Initial Table',
              'type': 'scatter3d',
              'uid': '84d17792-8963-418d-b513-a082a6896e9f',
              'x': array([ 1.92836283, -1.92836283,  1.02606043,  2.95442326, -2.95442326,
                          -1.02606043,  0.        ,  0.        ]),
              'y': array([ 2.29813333,  2.29813333, -2.81907786,  0.52094453,  0.52094453,
                          -2.81907786,  0.        ,  0.        ]),
              'z': array([3., 3., 3., 3., 3., 3., 3., 0.])},
             {'customdata': [transformed_table],
              'line': {'color': 'red'},
              'marker': {'size': 5},
              'mode': 'markers',
              'name': 'Transformed Table',
              'type': 'scatter3d',
              'uid': '00dbc606-537c-47b3-ac06-8841e3677979',
              'x':

In [22]:
display(HBox([VBox(WIDGETS), text_widget]))